In [31]:
import xgboost as xgb; 
print(xgb.__version__)

1.6.2


In [59]:
import os
import glob
import pandas as pd
import pickle
#import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import model_inference


In [2]:
# Build a .py script that takes a snapshot date, loads a model artefact and make an inference and save to datamart

## set up pyspark session

In [61]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

## set up config

In [74]:
snapshot_date_str = "2017-01-04"
model_name = "reg_2018_04_01.pkl"

In [72]:
config = {}
config["snapshot_date_str"] = snapshot_date_str
config["snapshot_date"] = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
config["model_name"] = model_name
config["model_bank_directory"] = "model_bank/"
config["model_artefact_filepath"] = config["model_bank_directory"] + config["model_name"]

## load model artefact from model bank

In [75]:
# Load the model from the pickle file
with open(config["model_artefact_filepath"], 'rb') as file:
    model_artefact = pickle.load(file)

print("Model loaded successfully! " + config["model_artefact_filepath"])

Model loaded successfully! model_bank/reg_2018_04_01.pkl


## load feature store

In [ ]:
from functools import reduce
feature_location = "datamart/gold/feature_store/"
# Load CSV into DataFrame - connect to feature store
files_list = glob.glob(os.path.join(feature_location, '*.parquet'))
features_store_sdf = spark.read.option("header", "true").parquet(*files_list)
print("row_count for features:",features_store_sdf.count(),"\n")

# Filter out NA
features_store_sdf = features_store_sdf.drop("snapshot_date","concentration","avg_delay_rate","act_days_to_deliver","total_freight_value","avg_processing_time","same_state","total_volume_cm3","seller_city","seller_state")
#rows_with_nulls = features_store_sdf.filter(
    #reduce(lambda a, b: a | b, (col(c).isNull() for c in features_store_sdf.columns)))


#order_ids_to_drop = [row["order_id"] for row in rows_with_nulls.select("order_id").distinct().collect()]
#features_store_sdf = features_store_sdf.filter(~col("order_id").isin(order_ids_to_drop))

#Extract relevant features
#features_store_sdf = features_store_sdf.filter(col("order_status") == "delivered")
features_sdf = features_store_sdf.toPandas()
features_sdf = features_sdf.dropna(how='any')
features_store_sdf = features_store_sdf[features_store_sdf["order_status"] == "delivered"]
print("extracted features_sdf", features_sdf.count(), config["snapshot_date"])

features_sdf

row_count for features: 37 

extracted features_sdf order_id             37
order_status         37
total_qty            37
total_price          37
total_weight_g       37
total_density        37
delivery_distance    37
same_city            37
is_weekend           37
avg_rating            3
day_of_week          37
season               37
dtype: int64 2016-09-16 00:00:00


,order_id,order_status,total_qty,total_price,total_weight_g,total_density,delivery_distance,same_city,is_weekend,avg_rating,day_of_week,season
0,09f58c00f941827ab206de7796785e44,delivered,1,8.90,200.0,0.568182,12.302606,1,0,NaN,5,Summer
1,0bda8164c1a12b6a388ebec8559ee287,delivered,1,6.90,200.0,0.568182,10.536861,1,0,NaN,5,Summer
2,17fed53ba6dfef9b594ee2268642e2aa,delivered,1,9.90,200.0,0.568182,12.528518,1,0,NaN,5,Summer
3,1b694ef5b28d3e949a4f0ffeb2c9fcd6,delivered,1,9.90,200.0,0.568182,10.536861,1,0,NaN,5,Summer
4,205d7052a6505124d200f6fea6b423bc,delivered,1,9.90,200.0,0.568182,10.536861,1,0,NaN,5,Summer
5,2f9d791088532b218b41f892cc16073b,delivered,1,9.90,200.0,0.568182,15.043834,1,0,NaN,5,Summer
6,34bf4feda1e203af64692d97c6950c39,delivered,1,10.90,200.0,0.568182,6.443899,1,0,NaN,5,Summer
7,38bcb524e1c38c2c1b60600a80fc8999,delivered,1,2.90,200.0,0.568182,11.661939,1,0,NaN,5,Summer
8,3ce4038922670fead496aeae61a8a393,delivered,1,10.90,100.0,0.284091,10.536861,1,0,NaN,5,Summer
9,40599d3d28b75746952ded75566637b9,delivered,1,11.90,200.0,0.568182,10.536861,1,0,NaN,5,Summer


In [85]:
test = features_store_sdf.dropna()
test

DataFrame[order_id: string, order_status: string, total_qty: bigint, total_price: double, total_weight_g: double, total_density: double, delivery_distance: double, same_city: int, is_weekend: int, avg_rating: double, day_of_week: int, season: string]

Processing for modelling

In [29]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(features_sdf[['season']])  # Only fit on training data
encoded_feature = encoder.transform(features_sdf[['season']])
encoded_f = pd.DataFrame(encoded_feature, columns=encoder.get_feature_names_out(['season']), index=features_sdf.index)
features_sdf = pd.concat([features_sdf.drop(columns=['season']), encoded_f], axis=1)

features_sdf

,order_id,order_status,total_qty,total_price,total_weight_g,total_density,delivery_distance,same_city,is_weekend,avg_rating,avg_delay_rate,day_of_week,season_Summer
0,09f58c00f941827ab206de7796785e44,delivered,1,8.90,200.0,0.568182,12.302606,1,0,5.000000,0.000000,5,1.0
1,0bda8164c1a12b6a388ebec8559ee287,delivered,1,6.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0
2,17fed53ba6dfef9b594ee2268642e2aa,delivered,1,9.90,200.0,0.568182,12.528518,1,0,5.000000,0.000000,5,1.0
3,1b694ef5b28d3e949a4f0ffeb2c9fcd6,delivered,1,9.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0
4,205d7052a6505124d200f6fea6b423bc,delivered,1,9.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0
5,2f9d791088532b218b41f892cc16073b,delivered,1,9.90,200.0,0.568182,15.043834,1,0,5.000000,0.000000,5,1.0
6,34bf4feda1e203af64692d97c6950c39,delivered,1,10.90,200.0,0.568182,6.443899,1,0,5.000000,0.000000,5,1.0
7,38bcb524e1c38c2c1b60600a80fc8999,delivered,1,2.90,200.0,0.568182,11.661939,1,0,5.000000,0.000000,5,1.0
8,3ce4038922670fead496aeae61a8a393,delivered,1,10.90,100.0,0.284091,10.536861,1,0,5.000000,0.000000,5,1.0
9,40599d3d28b75746952ded75566637b9,delivered,1,11.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0


In [33]:
# Ensure all expected columns are present
expected_columns = ['season_Spring', 'season_Summer', 'season_Winter']

for col in expected_columns:
    if col not in features_sdf.columns:
        features_sdf[col] = 0


features_sdf

,order_id,order_status,total_qty,total_price,total_weight_g,total_density,delivery_distance,same_city,is_weekend,avg_rating,avg_delay_rate,day_of_week,season_Summer,season_Spring,season_Winter
0,09f58c00f941827ab206de7796785e44,delivered,1,8.90,200.0,0.568182,12.302606,1,0,5.000000,0.000000,5,1.0,0,0
1,0bda8164c1a12b6a388ebec8559ee287,delivered,1,6.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0,0,0
2,17fed53ba6dfef9b594ee2268642e2aa,delivered,1,9.90,200.0,0.568182,12.528518,1,0,5.000000,0.000000,5,1.0,0,0
3,1b694ef5b28d3e949a4f0ffeb2c9fcd6,delivered,1,9.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0,0,0
4,205d7052a6505124d200f6fea6b423bc,delivered,1,9.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0,0,0
5,2f9d791088532b218b41f892cc16073b,delivered,1,9.90,200.0,0.568182,15.043834,1,0,5.000000,0.000000,5,1.0,0,0
6,34bf4feda1e203af64692d97c6950c39,delivered,1,10.90,200.0,0.568182,6.443899,1,0,5.000000,0.000000,5,1.0,0,0
7,38bcb524e1c38c2c1b60600a80fc8999,delivered,1,2.90,200.0,0.568182,11.661939,1,0,5.000000,0.000000,5,1.0,0,0
8,3ce4038922670fead496aeae61a8a393,delivered,1,10.90,100.0,0.284091,10.536861,1,0,5.000000,0.000000,5,1.0,0,0
9,40599d3d28b75746952ded75566637b9,delivered,1,11.90,200.0,0.568182,10.536861,1,0,5.000000,0.000000,5,1.0,0,0


In [78]:
from pyspark.sql.types import NumericType
if features_sdf.empty == True:
    y_inference_pdf = features_sdf[['order_id','order_status']]
    y_inference_pdf['model_name'] = ''
    y_inference_pdf['model_predictions'] = ''
    print('y_inference', y_inference_pdf.shape[0])
else: 
    # prepare X_inference
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoder.fit(features_sdf[['season']])  # Only fit on training data
    encoded_feature = encoder.transform(features_sdf[['season']])
    encoded_f = pd.DataFrame(encoded_feature, columns=encoder.get_feature_names_out(['season']), index=features_sdf.index)
    features_sdf = pd.concat([features_sdf.drop(columns=['season']), encoded_f], axis=1)
    expected_columns = ['season_Spring', 'season_Summer', 'season_Winter']
    for col in expected_columns:
        if col not in features_sdf.columns:
            features_sdf[col] = 0
    features_pdf = features_sdf.select_dtypes(include='number')
    transformer_stdscaler = model_artefact["preprocessing_transformers"]["stdscaler"]
    X_inference = transformer_stdscaler.transform(features_pdf)
    print('X_inference', X_inference.shape[0])

X_inference 37


/opt/anaconda3/envs/mle_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


## model prediction inference

In [79]:
threshold = model_artefact['threshold']
threshold

0.24

In [80]:
# load model
model = model_artefact["model"]
threshold = model_artefact['threshold']

# predict model
y_inference = model.predict_proba(X_inference)[:, 1]

# prepare output
y_inference_pdf = features_sdf[["order_id", "order_status"]].copy()
y_inference_pdf["model_name"] = config["model_name"]
y_inference_pdf["model_predictions"] = y_inference.round(4)
y_inference_pdf["model_predictions"] = (y_inference_pdf["model_predictions"] > threshold).astype(int)

y_inference_pdf

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

## save model inference to datamart gold table

In [22]:
# create bronze datalake
gold_directory = f"datamart/gold/model_predictions/{config['model_name'][:-4]}/"
print(gold_directory)

if not os.path.exists(gold_directory):
    os.makedirs(gold_directory)

# save gold table - IRL connect to database to write
partition_name = config["model_name"][:-4] + "_predictions_" + snapshot_date_str.replace('-','_') + '.parquet'
filepath = gold_directory + partition_name
spark.createDataFrame(y_inference_pdf).write.mode("overwrite").parquet(filepath)
# df.toPandas().to_parquet(filepath,
#           compression='gzip')
print('saved to:', filepath)

datamart/gold/model_predictions/reg_2017_12_04/


saved to: datamart/gold/model_predictions/reg_2017_12_04/reg_2017_12_04_predictions_2016_10_10.parquet


## backfill

In [10]:
# set up config
snapshot_date_str = "2023-01-01"

start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

In [11]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)


In [12]:
for snapshot_date in dates_str_lst:
    print(snapshot_date)
    model_inference.main(snapshot_date, model_name)

2023-01-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 1, 1, 0, 0),
 'snapshot_date_str': '2023-01-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl


extracted features_sdf 8974 2023-01-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/
saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2023_01_01.parquet


---completed job---


2023-02-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 2, 1, 0, 0),
 'snapshot_date_str': '2023-02-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2023-02-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2023_02_01.parquet


---completed job---


2023-03-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 3, 1, 0, 0),
 'snapshot_date_str': '2023-03-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2023-03-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2023_03_01.parquet


---completed job---


2023-04-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 4, 1, 0, 0),
 'snapshot_date_str': '2023-04-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2023-04-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2023_04_01.parquet


---completed job---


2023-05-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2023, 5, 1, 0, 0),
 'snapshot_date_str': '2023-05-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2023-05-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


...

saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2024_09_01.parquet


---completed job---


2024-10-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2024, 10, 1, 0, 0),
 'snapshot_date_str': '2024-10-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl


extracted features_sdf 8974 2024-10-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2024_10_01.parquet


---completed job---


2024-11-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2024, 11, 1, 0, 0),
 'snapshot_date_str': '2024-11-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2024-11-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2024_11_01.parquet


---completed job---


2024-12-01


---starting job---


{'model_artefact_filepath': 'model_bank/credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2024, 12, 1, 0, 0),
 'snapshot_date_str': '2024-12-01'}
Model loaded successfully! model_bank/credit_model_2024_09_01.pkl
extracted features_sdf 8974 2024-12-01 00:00:00
X_inference 8974
datamart/gold/model_predictions/credit_model_2024_09_01/


saved to: datamart/gold/model_predictions/credit_model_2024_09_01/credit_model_2024_09_01_predictions_2024_12_01.parquet


---completed job---




## Check datamart

In [29]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

In [31]:
folder_path = "scripts/datamart/gold/model_predictions/credit_model_2024_06_01/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 485
+-----------+-------------+--------------------+--------------------+
|customer_id|snapshot_date|          model_name|   model_predictions|
+-----------+-------------+--------------------+--------------------+
|  CUS_0x405|   2024-01-01|credit_model_2024...| 0.15490520000457764|
| CUS_0x4655|   2024-01-01|credit_model_2024...| 0.07329188287258148|
| CUS_0x4953|   2024-01-01|credit_model_2024...|  0.7080382704734802|
| CUS_0x4a0d|   2024-01-01|credit_model_2024...| 0.05151388421654701|
|  CUS_0x4d3|   2024-01-01|credit_model_2024...|  0.8074637651443481|
| CUS_0x4d40|   2024-01-01|credit_model_2024...|  0.6069353818893433|
| CUS_0x50e9|   2024-01-01|credit_model_2024...|  0.2434036284685135|
| CUS_0x54d7|   2024-01-01|credit_model_2024...|0.033143628388643265|
| CUS_0x57e5|   2024-01-01|credit_model_2024...| 0.21770469844341278|
| CUS_0x58cd|   2024-01-01|credit_model_2024...| 0.21139536798000336|
| CUS_0x5e61|   2024-01-01|credit_model_2024...|0.048095766454935074|
| CUS